In [1]:
from policyengine_us import Microsimulation
from policyengine_core.reforms import Reform
import pandas as pd

In [ ]:
baseline = Microsimulation()
baseline_net_income = baseline.calculate("household_net_income", period = 2024)
baseline_in_poverty = baseline.calculate("in_poverty", map_to="household")


In [ ]:
def calculate_changes_for_program():
    class NeutralizeProgram(Reform):
        def apply(self):
            self.neutralize_variable("nj_ctc")

    # Initialize the baseline and reformed simulations
    reformed = Microsimulation(reform = NeutralizeProgram)
    baseline.macro_cache_read = False
    reformed.macro_cache_read = False

    # Calculate net income change for the specific state
    state_codes = baseline.calculate("state_code")
    reformed_net_income = reformed.calculate("household_net_income", period = 2024)
    net_income_change = reformed_net_income[state_codes == "NJ"].sum() - baseline_net_income[state_codes == "NJ"].sum()


    # Calculate in_poverty mean for persons for the specific state
    reformed_in_poverty = reformed.calculate("in_poverty", map_to="household")
    baseline_in_poverty_mean = baseline_in_poverty[state_codes == "NJ"].mean()
    reformed_in_poverty_mean = reformed_in_poverty[state_codes == "NJ"].mean()
    in_poverty_mean_change = reformed_in_poverty_mean - baseline_in_poverty_mean

    # Calculate Gini index for the specific state
    personal_hh_equiv_income = reformed.calculate("equiv_household_net_income")
    household_count_people = reformed.calculate("household_count_people")
    personal_hh_equiv_income.weights *= household_count_people
    gini_index = personal_hh_equiv_income.gini()

    return net_income_change, in_poverty_mean_change, gini_index

In [ ]:
# Create a DataFrame and store the results directly
results = [{
    'Program': "nj_ctc",
    'State': "NJ",
    'Net Income Change': calculate_changes_for_program()[0],
    'In Poverty Mean Change': calculate_changes_for_program()[1],
    'Gini Index': calculate_changes_for_program()[2]
}]

# Convert the results to a DataFrame
results_df = pd.DataFrame(results)

# Display the DataFrame
print(results_df)


  Program State  Net Income Change  In Poverty Mean Change  Gini Index
0  nj_ctc    NJ                0.0                     0.0    0.392699


In [ ]:
calculate_changes_for_program()

(0.0, 0.0, 0.3926993687183417)